# Example QTL
___

This notebook gives an example of QTL analysis using `BigRiverQTL.jl`.

## Data

In this example, we will use a dataset available from the `R/qtl2` package. Specifically, we will use the BXD dataset, which is obtained from the [GeneNetwork](https://genenetwork.org/) website.

You can download the BXD genotype data from the following link:
[Download BXD Genotype Data](https://raw.githubusercontent.com/rqtl/qtl2data/master/BXD/bxd.zip)


### Example - BXD 

In [ ]:
# Libraries
using BigRiverQTL
using Plots

#### Data

We assume that the data is stored in `..\data\BXD` directory.

In [ ]:
########
# Data #
########
data_dir = joinpath(@__DIR__, "../data/BXD/");
file = joinpath(data_dir, "bxd.json");

Load bxd data using the function `get_geneticstudydata()`: 

In [ ]:
# Load bxd data
data = get_geneticstudydata(file);

In [ ]:
# The current version of `BigRiverQTL` does not have imputation functions.
# Remove the  missing data
data = get_data_completecases(data);

In [ ]:
# Data types
# gmap contains 
# makers info 
gInfo = data.gmap;

# phenotype info 
pInfo = data.phenocov;
# phenotype values 
pheno = data.pheno.val;

# We can get the genotype matrix using the following command.
# For computing reasons, we need to convert the geno matrix in Float64.
# One way to do it is to multiply by 1.0
geno = reduce(hcat, data.geno.val).*1.0;

#### Preprocessing

In [ ]:
#################
# Preprocessing #
#################
traitID = 1112;
pheno_y = pheno[:, traitID];
pheno_y2 = ones(length(pheno_y));
idx_not_missing = findall(!ismissing, pheno_y)
pheno_y2[idx_not_missing] = pheno_y[idx_not_missing];

#### Kinship

In [ ]:
###########
# Kinship #
###########
kinship = kinship_gs(geno,.99);

#### Scan

In [ ]:
########
# Scan #
########

single_results_perms = scan(
	pheno_y2,
	geno,
	kinship;
	permutation_test = true,
	nperms = 1000,
);

#### Preprocessing

#### Plots

In [ ]:
#########
# Plots #
#########

# QTL plots
plot_QTL(single_results_perms, gInfo, mbColname = "Pos")
savefig(joinpath(@__DIR__, "..", "images", "QTL_thrs_example.png"))

In [ ]:
# Manhattan plots
plot_manhattan(single_results_perms, gInfo, mbColname = "Pos")
savefig(joinpath(@__DIR__, "..", "images", "manhattan_thrs_example.png"))